In [2]:
import numpy as np
from PIL import Image

In [3]:
def rle(mask):
    flat_img = (mask > 0.5).ravel()
    
    starts = np.zeros(flat_img.shape[0], dtype=np.bool)
    ends = np.zeros(flat_img.shape[0], dtype=np.bool)
    starts[1:] = (~flat_img[:-1]) & flat_img[1:]
    ends[:-1] = flat_img[:-1] & (~flat_img[1:])

    starts[0] = flat_img[0]
    ends[-1] = flat_img[-1]
    
    starts_ix = np.where(starts)[0] + 1  # because rle indexed from 1
    ends_ix = np.where(ends)[0] + 1
    lengths = ends_ix - starts_ix + 1
    return starts_ix, lengths

In [4]:
def rle_dumps(img):
    starts_ix, lengths = rle(img)
    return " ".join((str(s) + " " + str(l) for s, l in zip(starts_ix, lengths)))

In [5]:
def rle_loads(rle, shape=(1280, 1918)):
    mask = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    tokens = rle.split()
    assert(len(tokens) % 2 == 0)
    for i in range(0, len(tokens), 2):
        start = int(tokens[i]) - 1  # rle is indexed from 1
        length = int(tokens[i+1])
        mask[start:start+length] = True
    return mask.reshape(shape)

In [84]:
def rle_encode(mask):
    pixels = mask.ravel()
    maskp = np.hstack((np.array([False]), pixels))
    maskm = np.hstack((pixels, np.array([False])))
    runs = np.where(maskp != maskm)[0]
    return runs[::2] + 1, runs[1::2] - runs[::2]

In [85]:
def rle_dumpsb(mask):
    starts, lengths = rle_encode(mask)
    return " ".join((str(s) + " " + str(l) for s, l in zip(starts, lengths)))

In [86]:
mask = np.array(Image.open('../data/train_masks/00087a6bd4dc_01_mask.gif'), dtype=np.uint8)

In [87]:
%timeit rle_dumpsb(mask)

100 loops, best of 3: 3.53 ms per loop


In [88]:
%timeit rle_dumps(mask)

100 loops, best of 3: 9.17 ms per loop


In [90]:
rle_dumpsb(mask) == rle_dumps(mask)

True